
---


[documentação](https://pandas.pydata.org/docs/index.html) <br>
dataset: [fonte](https://www.kaggle.com/datasets/rtatman/lego-database?select=inventory_parts.csv)

---


>>[1. vamos começar por importar a biblioteca e os datasets](#scrollTo=_8wgdKckiu4e)

>>[2. vamos explorar o dataset dos sets de legos](#scrollTo=nhJe9XIjkkJR)

>>[3. vamos explorar o dataset dos temas](#scrollTo=-74arCL7k0TJ)

>>[4. vamos explorar o dataset dos inventários](#scrollTo=ewU2CX6Wk8zL)

>>[5. vamos explorar o dataset dos inventários das peças](#scrollTo=t0HBoR-qlG0d)



## 1.&nbsp;vamos começar por importar a biblioteca e os datasets

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df_sets = pd.read_csv('sets.csv')
df_themes = pd.read_csv('themes.csv')
df_inventories = pd.read_csv('inventories.csv')
df_inventory_parts = pd.read_csv('inventory_parts.csv')
df_parts = pd.read_csv('parts.csv')
df_part_categories = pd.read_csv('part_categories.csv')
df_colors = pd.read_csv('colors.csv')

## 2.&nbsp;vamos explorar o dataset dos sets de legos

In [ ]:
df_sets.info()
df_sets.head()
df_sets['num_parts'].value_counts().reset_index().plot.scatter(x='num_parts', y='count')

In [ ]:
df_counts = df_sets['num_parts'].value_counts().reset_index()
df_counts.columns = ['num_parts', 'count']

mean_val = df_counts['num_parts'].mean()
median_val = df_counts['num_parts'].median()
percentile_80 = np.percentile(df_counts['num_parts'], 80)

# vamos criar o gráfico
sns.set_theme(style="white")
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_counts, x='num_parts', y='count', color='blue', edgecolor='black', alpha=0.7)
plt.xscale("log")
plt.axvline(mean_val, color='green', linestyle='dashed', linewidth=2, label=f"Mean: {mean_val:.0f}")
plt.axvline(median_val, color='orange', linestyle='solid', linewidth=2, label=f"Median: {median_val:.0f}")
plt.axvline(percentile_80, color='red', linestyle='dotted', linewidth=2, label=f"80% Threshold: {percentile_80:.0f}")
plt.xlabel("Número de peças (log scale)", fontsize=12)
plt.ylabel("Número de sets", fontsize=12)
plt.title("Distribuição do número de peças nos sets (Log Scale)", fontsize=14, fontweight='bold')
plt.legend()


In [ ]:
sets_per_year = df_sets.groupby("year").size()

# vamos criar o gráfico
sns.set_theme(style="white")
plt.figure(figsize=(12, 6))
sns.lineplot(x=sets_per_year.index, y=sets_per_year.values, marker="o", color="blue")
plt.xlabel("Ano", fontsize=12)
plt.ylabel("Número de sets (log scale)", fontsize=12)
plt.title("Número de sets lançados por ano", fontsize=14, fontweight="bold")

In [ ]:
# vamos calcular a soma cumulativa
cumulative_sets = sets_per_year.cumsum()

# vamos calcular momentos-chave
total_sets = cumulative_sets.max()
thresholds = {
    "20%": total_sets * 0.2,
    "50%": total_sets * 0.5,
    "80%": total_sets * 0.8
}

# vamos perceber em que anos acontecem
milestone_years = {label: cumulative_sets[cumulative_sets >= value].index[0] for label, value in thresholds.items()}

# vamos criar o gráfico
sns.set_theme(style="white")
plt.figure(figsize=(12, 6))
sns.lineplot(x=cumulative_sets.index, y=cumulative_sets.values, marker="o", color="blue", label="Cumulative LEGO Sets")
for label, year in milestone_years.items():
    plt.axvline(year, color="red", linestyle="dotted", linewidth=2, label=f"{label} Reached in {year}")
    plt.text(year, cumulative_sets[year], f"{label}", color="black", fontsize=12, verticalalignment="bottom", horizontalalignment="right")
plt.xlabel("Ano", fontsize=12)
plt.ylabel("Número cumulativo de sets (log scale)", fontsize=12)
plt.title("Número cumulativo de sets lançados por ano", fontsize=14, fontweight="bold")
plt.legend()

In [ ]:
# vamos adicionar a coluna da década
df_sets['decade'] = (df_sets['year'] // 10) * 10

# vamos calcular o número de peças médio por set por década
avg_parts_per_decade = df_sets.groupby('decade')['num_parts'].mean()

# vamos criar o gráfico
sns.set_theme(style="white")
plt.figure(figsize=(12, 6))
sns.barplot(x=avg_parts_per_decade.index, y=avg_parts_per_decade.values, hue=avg_parts_per_decade.index, palette='Blues_d', legend=False)
plt.xlabel("Década", fontsize=12)
plt.ylabel("Número médio de peças  por set(log scale)", fontsize=12)
plt.title("Número médio de peças por set por década", fontsize=14, fontweight="bold")

## 3.&nbsp;vamos explorar o dataset dos temas

In [ ]:
df_themes.head()
df_themes.info()

In [ ]:
df_themes.nunique()

In [ ]:
id_count_per_parent = round(df_themes.value_counts(subset="parent_id", normalize=True).head(15)*100,2).reset_index()

# vamos criar o gráfico
sns.set_theme(style="white")
plt.figure(figsize=(12, 6))
sns.barplot(
    x=id_count_per_parent.proportion,
    y=id_count_per_parent.parent_id.astype(str),
    hue=id_count_per_parent.parent_id.astype(str),
    palette='Blues_r',
    legend=False,
    orient='h'
)
plt.xlabel("Parent ID", fontsize=12)
plt.ylabel("Número de Parent IDs", fontsize=12)
plt.title("Número de IDs por Parent ID (%)", fontsize=14, fontweight="bold")

In [ ]:
id_count_per_parent_names = id_count_per_parent.merge(
    df_themes[["id", "name"]],
    left_on='parent_id',
    right_on='id',
    how='left'
).drop(columns=['id']).rename(columns={'name': 'parent_name'})

# vamos criar o gráfico
sns.set_theme(style="white")
plt.figure(figsize=(12, 6))
sns.barplot(
    x=id_count_per_parent_names.proportion,
    y=id_count_per_parent_names.parent_name,
    hue=id_count_per_parent_names.parent_name,
    palette='Blues_r',
    legend=False,
    orient='h'
)
plt.xlabel("Parent ID", fontsize=12)
plt.ylabel("Número de Parent IDs", fontsize=12)
plt.title("Número de IDs por Parent ID (%)", fontsize=14, fontweight="bold")

## 4.&nbsp;vamos explorar o dataset dos inventários

In [ ]:
df_inventories.head()
df_inventories.info()

In [ ]:
# vamos verificar o número de valores únicos por coluna
df_inventories.nunique()

In [ ]:
# vamos contar o número de inventories_id para cada set
df_inventories_grouped = df_inventories.groupby("set_num").size().sort_values(ascending=False).reset_index()
df_inventories_grouped.columns = ["set_num", "num_inventories"]
df_inventories_grouped

# vamos considerar apenas os sets com mais do que um inventário
condition_1 = df_inventories_grouped["num_inventories"] > 1
df_inventories_grouped = df_inventories_grouped[condition_1]
df_inventories_grouped

# vamos ver o nome desses sets
df_inventories_grouped = df_inventories_grouped[["set_num"]].merge(df_sets[["set_num", "name"]], on="set_num", how="left").merge(df_inventories[["set_num", "id"]], on="set_num", how="left")
df_inventories_grouped

## 5.&nbsp;vamos explorar o dataset dos inventários das peças

In [ ]:
df_inventory_parts.head()
df_inventory_parts.info()

In [ ]:
# vamos verificar o número de peças por inventário, considerando que podem ser peças sobresselentes
df_pivot = df_inventory_parts.pivot_table(
    index="inventory_id",
    columns="is_spare",
    values="quantity",
    aggfunc="sum",
    fill_value=0,
    margins=True
).reset_index()
df_pivot.columns.name = None
df_pivot.columns = ["inventory_id", "not_spare", "spare", "total"]
df_pivot

In [ ]:
# vamos confirmar se a diferença dos inventários no mesmo set está no número de peças
df_inventories_grouped.merge(df_pivot, right_on="inventory_id", left_on="id", how="left").drop(columns=["id"])